In [ ]:
import os
import time
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from pytesseract import Output
import pytesseract
import cv2
import io

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

import urllib.request

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

from PIL import Image, ImageOps, ImageFilter
from skimage import io
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects

#텍스트 전처리
import re
import nltk
from nltk.corpus import stopwords

### colab에서 tesseract 사용 시
### !sudo apt install tesseract-ocr
### !pip install pytesseract
### !sudo apt install tesseract-ocr libtesseract-dev tesseract-ocr-kor (한국어)

## 이미지 크롤링 후 OCR 진행

In [ ]:
def initialize_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service('D:\chromedriver-win32\chromedriver.exe'), options=chrome_options)
    return driver

def get_theater_links(driver, query):
    driver.get(query)
    time.sleep(1)
    number = 160

    addresslist = []

    for i in range(1, int(number) + 1):
        xpath = f"/html/body/table/tbody/tr[2]/td[3]/div/div/div[2]/div/table/tbody/tr[{i}]/td[1]/a"
        elem = driver.find_element(By.XPATH, xpath).get_attribute("href")
        addresslist.append(elem)

    return addresslist


def resize_image(image, max_height):
    # 이미지 크기를 최대 높이에 맞춰 조정
    width_percent = (max_height / float(image.size[1]))
    new_width = int((float(image.size[0]) * float(width_percent)))
    resized_image = image.resize((new_width, max_height), Image.LANCZOS)
    return resized_image

def image(driver, addresslist, save_directory, max_image_height=2000):
    scraped_image = []
    for index, lis in enumerate(addresslist):
        driver.get(lis)
        time.sleep(1)

        # 이미지 요소 찾기
        img_area_elements = driver.find_elements(By.XPATH, '//*[@id="productMainBody"]/div/div/div[5]/div/p/strong/img')
        title_element = driver.find_element(By.XPATH, '//*[@id="container"]/div[5]/div[1]/div[2]/div[1]/div/div[1]/h2')

        # 이미지가 없을 때 처리
        if not img_area_elements:
            img_area_elements = driver.find_elements(By.XPATH, '//*[@id="productMainBody"]/div/div/div[3]/div/p/strong/img')
            if not img_area_elements:
                img_area_elements = driver.find_elements(By.XPATH, '//*[@id="productMainBody"]/div/div/div[4]/div/p/strong/img')
            # 이미지 저장
            image_list = []
            for i, img_element in enumerate(img_area_elements):
                img_url = img_element.get_attribute("src")
                img_data = urllib.request.urlopen(img_url).read()

                # 파일 이름 생성
                filename=title_element.text.replace('/', '_').replace(':', '_')
                file_name= filename+"_part"+str(i + 1)+".png"
                file_path = os.path.join(save_directory, file_name)

                # 이미지 저장
                with open(file_path, "wb") as file:
                    file.write(img_data)
                    print(f"Saved {file_name}")

        else:
            # 이미지가 있는 경우 처리 코드

            # 이미지 저장
            image_list = []
            for i, img_element in enumerate(img_area_elements):
                img_url = img_element.get_attribute("src")
                img_data = urllib.request.urlopen(img_url).read()

                # 파일 이름 생성
                filename=title_element.text.replace('/', '_').replace(':', '_')
                file_name= filename+"_part"+str(i + 1)+".png"
                file_path = os.path.join(save_directory, file_name)

                # 이미지 저장
                with open(file_path, "wb") as file:
                    file.write(img_data)
                    print(f"Saved {file_name}")

        #7.이미지에서 문자추출
        for i, img_element in enumerate(img_area_elements):
            img_text=[]
            img_name='이미지 저장 경로/'+filename+"_part"+str(i + 1)+".png"
            img1=np.array(Image.open(img_name))
            text=pytesseract.image_to_string(img1,config='--psm 6 --oem 1',lang='kor')
            img_text.append(text)
            #img_text.append('\n')

            scraped_image.append({'Title': title_element.text, 'Text':text}) #,'img_URL': img_url})
            #print(text)

    return scraped_image

def save_to_csv(data_list, save_path):
    if data_list:
        combined_data = pd.DataFrame(data_list)
        combined_data.to_csv(save_path, index=False)

In [ ]:
current_directory = os.getcwd()
save_path = os.path.join(current_directory, "제목/저장경로/.csv")

driver = initialize_driver()
query = "http://ticket.interpark.com/TPGoodsList.asp?Ca=Dra&Sort=3"
addresslist = get_theater_links(driver, query)
scraped_image = image(driver, addresslist,'포스터 저장경로/')
save_to_csv(scraped_image, save_path)
driver.quit()

Saved 연극 너의 목소리가 들려_part1.png
Saved 연극 너의 목소리가 들려_part2.png
Saved 연극 너의 목소리가 들려_part3.png
Saved 대학로 1위 연극 〈쉬어매드니스〉_part1.png
Saved 대학로 1위 연극 〈쉬어매드니스〉_part2.png
Saved 행오버_part1.png
Saved 행오버_part2.png
Saved 행오버_part3.png
Saved 행오버_part4.png
Saved 연극 〈바닷마을 다이어리〉_part1.png
Saved 연극 〈바닷마을 다이어리〉_part2.png
Saved 연극 한뼘사이_part1.png
Saved 연극 한뼘사이_part2.png
Saved 연극 한뼘사이_part3.png
Saved 연극 한뼘사이_part4.png
Saved 연극 한뼘사이_part5.png
Saved 연극 한뼘사이_part6.png
Saved 연극 한뼘사이_part7.png
Saved 연극 한뼘사이_part8.png
Saved 연극 한뼘사이_part9.png
Saved 연극 한뼘사이_part10.png
Saved 연극 한뼘사이_part11.png
Saved 연극 한뼘사이_part12.png
Saved 연극 한뼘사이_part13.png
Saved 연극 한뼘사이_part14.png
Saved 연극 한뼘사이_part15.png
Saved 연극〈늘근도둑이야기〉_part1.png
Saved 연극〈늘근도둑이야기〉_part2.png
Saved 뮤직드라마 〈불편한 편의점〉_part1.png
Saved 뮤직드라마 〈불편한 편의점〉_part2.png
Saved 뮤직드라마 〈불편한 편의점〉_part3.png
Saved 뮤직드라마 〈불편한 편의점〉_part4.png
Saved 뮤직드라마 〈불편한 편의점〉_part5.png
Saved 코믹감동 휴먼판타지 연극 〈2호선 세입자〉_part1.png
Saved 코믹감동 휴먼판타지 연극 〈2호선 세입자〉_part2.png
Saved 10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀_part